In [1]:
from deep_sort_realtime.deepsort_tracker import DeepSort
import tensorflow as tf
import cv2

import numpy as np

In [2]:
# 모델 불러오기
model = tf.saved_model.load('../../../Data/COCO-SSD_Mobilenet_V2')

In [3]:
# 2. 로컬 영상 파일 열기
video_path = '../../../Data/video/1-1_608-C05.mp4'
cap = cv2.VideoCapture(video_path)

In [6]:
# 3. 프레임별 처리
tracker = DeepSort(max_age=30, nn_budget=70)  # DeepSORT 초기화

object_tracks = {}  # 객체 위치 저장용 딕셔너리
TARGET_CLASS = 1
count_num = 0

frame_count = 0
skip_frames = 5  # 5프레임마다 한 번씩 처리

print("영상시작")

# 첫 번째 프레임에서 프레임 크기 초기화
ret, frame = cap.read()
if not ret:  # 첫 번째 프레임 읽기 실패 시 프로그램 종료
    print("비디오 파일을 읽을 수 없습니다.")
    cap.release()
    exit()

frame_height, frame_width = frame.shape[:2]
mid_x = frame_width // 2  # 화면 중간 x좌표 계산

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    if frame_count % skip_frames != 0:  # 프레임 스킵
        continue
    
    # OpenCV BGR -> RGB 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    input_tensor = tf.convert_to_tensor(rgb_frame)
    input_tensor = input_tensor[tf.newaxis, ...]

    # 객체 탐지 실행
    detections = model(input_tensor)
    detection_boxes = detections['detection_boxes'][0].numpy()
    detection_classes = detections['detection_classes'][0].numpy()
    detection_scores = detections['detection_scores'][0].numpy()

    # 현재 프레임에서 탐지된 객체 정보 저장
    current_frame_tracks = {}

    # 탐지 결과 표시
    dets = []

    for i in range(len(detection_scores)):
        if detection_scores[i] > 0.5 and detection_classes[i] == TARGET_CLASS:  # 객체 분류에 대한 신뢰도 임계값 0.5 이상으로 설정

            box = detection_boxes[i] * [frame.shape[0], frame.shape[1], frame.shape[0], frame.shape[1]]
            ymin, xmin, ymax, xmax = box.astype('int')
            dets.append([float(xmin), float(ymin), float(xmax), float(ymax), float(detection_scores[i])])  # 형식 변환

            # DeepSORT 추적 실행
            if dets:  # dets가 비어 있지 않은 경우에만 호출
                print("Detections:", dets)  # 디버깅 출력
                
                # dets 데이터를 올바른 형식으로 변환
                formatted_dets = [[d[:4], d[4]] for d in dets]  # 2D좌표(Bbox), score
                tracks = tracker.update_tracks(formatted_dets, frame=frame)

            # 추적 결과 처리
            for track in tracks:
                if not track.is_confirmed() or track.time_since_update > 1:
                    continue

                track_id = track.track_id  # 객체 ID
                l, t, r, b = track.to_ltrb()  # 추적된 경계 상자 좌표
                center_x = int((l + r) / 2)

                # 왼쪽 -> 오른쪽으로 이동
                if center_x < mid_x and center_x >= mid_x:
                    count_num += 1
                    print(f"객체 {track_id}가 왼쪽에서 오른쪽으로 이동. 현재 인원수: {count_num}")

                # 오른쪽 -> 왼쪽으로 이동
                elif center_x > mid_x and center_x <= mid_x:
                    count_num -= 1
                    print(f"객체 {track_id}가 오른쪽에서 왼쪽으로 이동. 현재 인원수: {count_num}")

                # 경계 상자 및 ID 시각화
                cv2.rectangle(frame, (int(l), int(t)), (int(r), int(b)), (0, 255, 0), 2)
                cv2.putText(frame, f'ID: {track_id}', (int(l), int(t) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    
    
    # 영상 중간 부분에 세로선 그리기
    cv2.line(frame, (mid_x, 0), (mid_x, frame_height), (255, 0, 0), 2)  # (255, 0, 0)은 파란색, 두께는 2

    # 프레임 보여주기
    cv2.imshow('Detection with Direction', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


영상시작
Detections: [[663.0, 253.0, 916.0, 853.0, 0.6391046643257141]]
Detections: [[711.0, 230.0, 946.0, 828.0, 0.6060701608657837]]
Detections: [[711.0, 230.0, 946.0, 828.0, 0.6060701608657837], [769.0, 244.0, 923.0, 706.0, 0.5302690863609314]]
Detections: [[870.0, 235.0, 1113.0, 828.0, 0.6295047998428345]]
Detections: [[870.0, 235.0, 1113.0, 828.0, 0.6295047998428345], [900.0, 232.0, 1066.0, 685.0, 0.5551256537437439]]
Detections: [[975.0, 231.0, 1118.0, 690.0, 0.5375898480415344]]
Detections: [[975.0, 231.0, 1118.0, 690.0, 0.5375898480415344], [939.0, 213.0, 1151.0, 830.0, 0.524494469165802]]
Detections: [[994.0, 215.0, 1206.0, 822.0, 0.5405222773551941]]
Detections: [[1046.0, 218.0, 1291.0, 822.0, 0.5152407884597778]]
Detections: [[1083.0, 209.0, 1315.0, 796.0, 0.6488387584686279]]
Detections: [[1175.0, 216.0, 1333.0, 645.0, 0.6329004764556885]]
Detections: [[1175.0, 216.0, 1333.0, 645.0, 0.6329004764556885], [1147.0, 196.0, 1356.0, 785.0, 0.5625035166740417]]
Detections: [[1183.0, 2